In [2]:
# !wget https://huggingface.co/datasets/malaysia-ai/starcoderdata-sample/resolve/main/ruby.jsonl

In [3]:
from huggingface_hub import InferenceClient
from tqdm import tqdm
import os
import json
from glob import glob

/home/husein/.local/lib/python3.8/site-packages/requests/__init__.py:102: RequestsDependencyWarning: urllib3 (1.26.15) or chardet (5.2.0)/charset_normalizer (2.0.7) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({})/charset_normalizer ({}) doesn't match a supported "


In [4]:
ls = []

with open('ruby.jsonl') as fopen:
    for l in tqdm(fopen):
        try:
            l = json.loads(l)
            if len(l['content'].split()) > 5e3:
                continue
            ls.append(l['content'])
        except:
            pass
len(ls)

100000it [00:04, 20361.58it/s]


99354

In [5]:
client = InferenceClient(
    "", timeout = 120
)


def format_prompt(message, history):
  prompt = "<s>"
  for user_prompt, bot_response in history:
    prompt += f"[INST] {user_prompt} [/INST]"
    prompt += f" {bot_response}</s> "
  prompt += f"[INST] {message} [/INST]"
  return prompt

def format_user(history):
    prompt = "<s>"
    for user_prompt, bot_response in history:
        prompt += f"[INST] {user_prompt} [/INST]"
        prompt += f" {bot_response}</s> "
    prompt += f"[INST]"
    return prompt

In [6]:
generate_kwargs = dict(
    temperature=1.0,
    max_new_tokens=5120,
    top_p=0.95,
    repetition_penalty=1.0,
    do_sample=True,
)

In [8]:
!mkdir mixtral-ruby
# !rm -rf mixtral-ruby/*.json

mkdir: cannot create directory ‘mixtral-ruby’: File exists


In [9]:
def answer(q, i):
    filename = f'mixtral-ruby/{i}.json'
    if os.path.exists(filename):
        return
    
    while True:
    # for _ in range(5):
        try:
            
            prompt = f"""
Please gain inspiration from the following random code snippet to create a high-quality Ruby problem. Present your output in two distinct sections: [Problem Description] and [Solution].

Ruby snippet for inspiration:
```
{q}
```

Guidelines for each section:

1. [Problem Description]: This should be **completely self-contained**, providing all the contextual information one needs to understand and solve the problem. Assume common programming knowledge, but ensure that any specific context, variables, or code snippets pertinent to this problem are explicitly included but do not mention code snippet above.

2. [Solution]: Offer a comprehensive, **correct** solution that accurately addresses the [Problem Description] you provided.
""".strip()
            formatted_prompt = format_prompt(prompt, [])
            stream = client.text_generation(formatted_prompt, **generate_kwargs, stream=False, details=True, return_full_text=False)
            output = stream.generated_text
            with open(filename, 'w') as fopen:
                json.dump(output, fopen)
            break
        except Exception as e:
            if 'tokens + `max_new_tokens`' in str(e):
                with open(filename, 'w') as fopen:
                    json.dump(False, fopen)
                break
            pass

In [10]:
def consumer(queue, name):
    while True:
        if queue.qsize() == 0:
            break
        item = queue.get()
        answer(*item)
    print(f'consumer {name} done')

In [11]:
from threading import Thread
from queue import Queue

queue = Queue()
questions = ls
urls = [(q, no) for no, q in enumerate(questions)]
for u in urls:
    queue.put(u)
    
ori_size = queue.qsize()

In [12]:
answer(*urls[1])

In [13]:
with open('mixtral-ruby/1.json') as fopen:
    print(json.loads(fopen.read()))

 **Problem Description**

---

Bosh is an open-source tool for release engineering, deployment, lifecycle management, and troubleshooting of large scale distributed services. Bosh uses stemcells as the underlying operating system and environment upon which deployed software runs. As a part of the Bosh ecosystem, you are required to build and test a new stemcell with updated software packages. You will write a Ruby class `StemcellBuilder` to automate the process of downloading an existing stemcell, extracting its contents, updating specific packages, and creating a new stemcell.

The `StemcellBuilder` class should provide the following functionalities:

1. Accept an input file path to a valid Bosh stemcell archive.
2. Extract the stemcell archive into a temporary directory.
3. Accept an array of package names that should be updated. These package names are already available in a file named `packages.txt` located in the root directory of the extracted stemcell.
4. Update the specified pa

In [ ]:
max_worker = 50
consumers = [Thread(target=consumer, args=(queue,i)) for i in range(max_worker)]
for i in range(len(consumers)):
    consumers[i].start()
    
pbar = tqdm(total=ori_size)
last_size = 0
while True:
    size = queue.qsize()
    if size == 0:
        break
    left = ori_size - size
    minus = left - last_size
    if minus > 0:
        pbar.update(minus)
        last_size += minus

pbar.close()

 12%|███████████▉                                                                                      | 12133/99354 [13:38:28<19:57:46,  1.21it/s]